# Performance Optimization: JAX Compilation and GPU Acceleration

Master JAX JIT compilation, GPU acceleration, memory optimization, and NLSQ performance for rheological computations.

## Learning Objectives
- Profile code to identify performance bottlenecks
- Understand JAX JIT compilation benefits and overhead
- Benchmark CPU vs GPU performance for large datasets
- Optimize memory usage for large-scale computations
- Leverage NLSQ for 5-270x optimization speedup
- Apply vectorization with jax.vmap for batch operations
- Scale workflows to 100K+ data points efficiently

## Prerequisites
- JAX basics (safe imports, array operations)
- Model fitting experience (Maxwell, fractional models)
- Basic understanding of NumPy/SciPy optimization

**Estimated Time:** 55-60 minutes

## Setup and Environment Detection

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import matplotlib


import numpy as np
import time
import tracemalloc
import matplotlib.pyplot as plt
from IPython.display import display
from rheojax.core.jax_config import safe_import_jax
from rheojax.models.maxwell import Maxwell
from rheojax.models.fractional_maxwell_liquid import FractionalMaxwellLiquid
from rheojax.utils.optimization import nlsq_optimize
import warnings
warnings.filterwarnings('ignore')

# Safe JAX import (enforces float64 precision)
jax, jnp = safe_import_jax()

print('='*60)
print('JAX Environment Information')
print('='*60)
print(f'JAX version: {jax.__version__}')
print(f'Available devices: {jax.devices()}')
print(f'Default backend: {jax.default_backend()}')
print(f'Float64 enabled: {jax.config.jax_enable_x64}')
print('='*60)

# Detect GPU availability
HAS_GPU = jax.default_backend() == 'gpu'
if HAS_GPU:
    print('✓ GPU acceleration available')
else:
    print('⚠ CPU-only mode (see CLAUDE.md for GPU installation)')

# Suppress matplotlib backend warning in VS Code
warnings.filterwarnings('ignore', message='.*non-interactive.*')


## 1. Performance Profiling: Establishing Baselines

**Duration:** ~5 minutes

Before optimizing, measure baseline performance to identify bottlenecks.

In [ ]:
# Utility: Simple timing decorator
def benchmark(func, *args, n_runs=10, warmup=2, **kwargs):
    """Benchmark function with warmup runs."""
    # Warmup
    for _ in range(warmup):
        _ = func(*args, **kwargs)
    
    # Measure
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        times.append(elapsed)
    
    return {
        'mean': np.mean(times),
        'std': np.std(times),
        'min': np.min(times),
        'max': np.max(times),
        'result': result
    }

# Generate test dataset
N = 1000
t_test = np.logspace(-2, 2, N)
G_test = 1e5 * np.exp(-t_test / 0.1) + np.random.normal(0, 1e3, N)

# Baseline: Fit Maxwell model
print('Baseline Performance (N=1000 points)')
print('-' * 60)

model = Maxwell()
stats = benchmark(model.fit, t_test, G_test, n_runs=5)

print(f"Model fitting: {stats['mean']*1000:.2f} ± {stats['std']*1000:.2f} ms")
print(f"  Min: {stats['min']*1000:.2f} ms")
print(f"  Max: {stats['max']*1000:.2f} ms")
print(f"  Per-point: {stats['mean']/N*1e6:.2f} µs")

### Memory Profiling

Track memory allocation during model fitting.

In [ ]:
# Memory profiling with tracemalloc
tracemalloc.start()

# Baseline memory snapshot
snapshot_before = tracemalloc.take_snapshot()

# Perform operation
model = Maxwell()
model.fit(t_test, G_test)
predictions = model.predict(t_test)

# Measure memory usage
snapshot_after = tracemalloc.take_snapshot()
top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print('\nMemory Usage (Baseline)')
print('-' * 60)
print(f"Current: {current / 1024 / 1024:.2f} MB")
print(f"Peak: {peak / 1024 / 1024:.2f} MB")
print(f"Per-point: {peak / N / 1024:.2f} KB")

## 2. JAX JIT Compilation: Compilation Overhead vs Speedup

**Duration:** ~8 minutes

JAX's Just-In-Time (JIT) compilation compiles Python/NumPy code to optimized XLA. 
- **First call:** Slow (compilation overhead)
- **Subsequent calls:** Fast (cached compiled code)

In [ ]:
# Define Maxwell relaxation modulus (non-jitted)
def maxwell_modulus_plain(params, t):
    """Plain Python implementation."""
    G0, eta = params
    tau = eta / G0
    return G0 * jnp.exp(-t / tau)

# JIT-compiled version
@jax.jit
def maxwell_modulus_jit(params, t):
    """JIT-compiled implementation."""
    G0, eta = params
    tau = eta / G0
    return G0 * jnp.exp(-t / tau)

# Test data
t_jax = jnp.logspace(-2, 2, 5000)
params = jnp.array([1e5, 1e3])

print('JIT Compilation Overhead Analysis')
print('='*60)

# First call: compilation overhead
start = time.perf_counter()
result_first = maxwell_modulus_jit(params, t_jax)
time_first = time.perf_counter() - start

print(f"First call (compile + execute): {time_first*1000:.2f} ms")

# Subsequent calls: cached compiled code
times_cached = []
for _ in range(100):
    start = time.perf_counter()
    _ = maxwell_modulus_jit(params, t_jax)
    times_cached.append(time.perf_counter() - start)

time_cached_mean = np.mean(times_cached)
print(f"Cached calls (mean): {time_cached_mean*1000:.3f} ms")
print(f"Speedup after compilation: {time_first/time_cached_mean:.0f}x")

# Compare with non-jitted version
times_plain = []
for _ in range(100):
    start = time.perf_counter()
    _ = maxwell_modulus_plain(params, t_jax)
    times_plain.append(time.perf_counter() - start)

time_plain_mean = np.mean(times_plain)
print(f"\nNon-jitted (mean): {time_plain_mean*1000:.3f} ms")
print(f"JIT speedup: {time_plain_mean/time_cached_mean:.1f}x")

### JIT Compilation Best Practices

**When to use JIT:**
- ✓ Functions called repeatedly (amortize compilation cost)
- ✓ Numerical loops and array operations
- ✓ Core optimization routines

**When to avoid JIT:**
- ✗ One-off operations (compilation overhead > execution)
- ✗ Functions with dynamic shapes (triggers recompilation)
- ✗ I/O operations or Python side effects

In [ ]:
# Visualize JIT compilation overhead vs speedup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Left: Time per call
# Ensure arrays have consistent lengths
n_calls = min(100, len(times_cached) + 1, len(times_plain))
calls = np.arange(1, n_calls + 1)
time_jit = np.concatenate([[time_first], times_cached[:n_calls-1]])
time_plain_all = np.array(times_plain[:n_calls])

ax1.plot(calls, time_jit * 1000, label='JIT', linewidth=2)
ax1.plot(calls, time_plain_all * 1000, label='Plain', linewidth=2, alpha=0.7)
ax1.axvline(1, color='red', linestyle='--', alpha=0.5, label='Compilation')
ax1.set_xlabel('Call Number')
ax1.set_ylabel('Time (ms)')
ax1.set_title('JIT Compilation Overhead')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Cumulative time
cumulative_jit = np.cumsum(time_jit)
cumulative_plain = np.cumsum(time_plain_all)

ax2.plot(calls, cumulative_jit, label='JIT', linewidth=2)
ax2.plot(calls, cumulative_plain, label='Plain', linewidth=2, alpha=0.7)
ax2.set_xlabel('Call Number')
ax2.set_ylabel('Cumulative Time (s)')
ax2.set_title('Total Execution Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

# Break-even point
breakeven = np.argmax(cumulative_jit < cumulative_plain)
print(f"\nBreak-even point: {breakeven} calls")
print(f"After {breakeven} calls, JIT is faster overall")

## 3. GPU Acceleration: CPU vs GPU Benchmarks

**Duration:** ~10 minutes

Compare CPU and GPU performance across different dataset sizes.

In [ ]:
# Dataset sizes to benchmark
sizes = [100, 500, 1000, 5000, 10000, 50000, 100000]

results = {'size': [], 'cpu_time': [], 'gpu_time': [], 'speedup': []}

print('CPU vs GPU Performance Benchmark')
print('='*60)
print(f"{'Size':<10} {'CPU (ms)':<12} {'GPU (ms)':<12} {'Speedup':<10}")
print('-'*60)

for N in sizes:
    # Generate dataset
    t = np.logspace(-2, 2, N)
    G_true = 1e5 * np.exp(-t / 0.1)
    G_data = G_true + np.random.normal(0, 1e3, N)
    
    # CPU timing
    model_cpu = Maxwell()
    with jax.default_device(jax.devices('cpu')[0]):
        stats_cpu = benchmark(model_cpu.fit, t, G_data, n_runs=3, warmup=1)
    
    cpu_time = stats_cpu['mean'] * 1000  # Convert to ms
    
    # GPU timing (if available)
    if HAS_GPU:
        model_gpu = Maxwell()
        with jax.default_device(jax.devices('gpu')[0]):
            stats_gpu = benchmark(model_gpu.fit, t, G_data, n_runs=3, warmup=1)
        gpu_time = stats_gpu['mean'] * 1000
        speedup = cpu_time / gpu_time
    else:
        gpu_time = np.nan
        speedup = 1.0
    
    # Store results
    results['size'].append(N)
    results['cpu_time'].append(cpu_time)
    results['gpu_time'].append(gpu_time)
    results['speedup'].append(speedup)
    
    # Print results
    if HAS_GPU:
        print(f"{N:<10} {cpu_time:<12.2f} {gpu_time:<12.2f} {speedup:<10.1f}x")
    else:
        print(f"{N:<10} {cpu_time:<12.2f} {'N/A':<12} {'-':<10}")

print('='*60)

### Visualize Scaling Performance

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Absolute timing
ax1.loglog(results['size'], results['cpu_time'], 'o-', label='CPU', linewidth=2, markersize=8)
if HAS_GPU:
    ax1.loglog(results['size'], results['gpu_time'], 's-', label='GPU', linewidth=2, markersize=8)
ax1.set_xlabel('Dataset Size (N points)', fontsize=12)
ax1.set_ylabel('Execution Time (ms)', fontsize=12)
ax1.set_title('CPU vs GPU Scaling', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')

# Right: Speedup factor
if HAS_GPU:
    ax2.semilogx(results['size'], results['speedup'], 'o-', linewidth=2, markersize=8, color='green')
    ax2.axhline(1.0, color='red', linestyle='--', alpha=0.5, label='No speedup')
    ax2.set_xlabel('Dataset Size (N points)', fontsize=12)
    ax2.set_ylabel('GPU Speedup Factor', fontsize=12)
    ax2.set_title('GPU Acceleration vs Dataset Size', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'GPU not available\n(CPU-only mode)', 
             ha='center', va='center', fontsize=14, transform=ax2.transAxes)
    ax2.axis('off')

plt.tight_layout()
display(fig)
plt.close(fig)

# Summary statistics
if HAS_GPU:
    max_speedup = np.max(results['speedup'])
    max_speedup_size = results['size'][np.argmax(results['speedup'])]
    print(f"\nMaximum GPU speedup: {max_speedup:.1f}x at N={max_speedup_size}")
    print(f"Average speedup (N≥10K): {np.mean([s for s, n in zip(results['speedup'], results['size']) if n >= 10000]):.1f}x")


### Data Transfer Overhead Analysis

GPU acceleration includes overhead from CPU↔GPU data transfer.

In [ ]:
if HAS_GPU:
    print('GPU Data Transfer Overhead')
    print('='*60)
    
    N = 50000
    t_cpu = np.logspace(-2, 2, N)
    G_cpu = 1e5 * np.exp(-t_cpu / 0.1)
    
    # Measure transfer time
    start = time.perf_counter()
    t_gpu = jax.device_put(t_cpu, jax.devices('gpu')[0])
    G_gpu = jax.device_put(G_cpu, jax.devices('gpu')[0])
    transfer_time = time.perf_counter() - start
    
    # Measure computation time
    @jax.jit
    def gpu_compute(t, G):
        return jnp.sum(G * jnp.exp(-t))
    
    # Warmup
    _ = gpu_compute(t_gpu, G_gpu)
    
    start = time.perf_counter()
    result = gpu_compute(t_gpu, G_gpu)
    compute_time = time.perf_counter() - start
    
    # Transfer back
    start = time.perf_counter()
    _ = np.array(result)
    transfer_back_time = time.perf_counter() - start
    
    total_overhead = transfer_time + transfer_back_time
    
    print(f"Dataset size: N={N}")
    print(f"CPU→GPU transfer: {transfer_time*1000:.3f} ms")
    print(f"GPU computation: {compute_time*1000:.3f} ms")
    print(f"GPU→CPU transfer: {transfer_back_time*1000:.3f} ms")
    print(f"Total overhead: {total_overhead*1000:.3f} ms")
    print(f"Overhead ratio: {total_overhead/compute_time:.1f}x computation time")
    print('\n⚠ Keep data on GPU for repeated operations to minimize overhead')
else:
    print('GPU not available - skipping transfer overhead analysis')

## 4. Memory Optimization: Profiling and Reduction

**Duration:** ~8 minutes

Optimize memory usage for large datasets.

### Float32 vs Float64 Memory Tradeoff

Float32 uses half the memory but may lose precision for rheological calculations.

In [ ]:
N_large = 100000
t_large = np.logspace(-2, 2, N_large)
G_large = 1e5 * np.exp(-t_large / 0.1)

# Float64 (default)
t_f64 = jnp.array(t_large, dtype=jnp.float64)
G_f64 = jnp.array(G_large, dtype=jnp.float64)

# Float32
t_f32 = jnp.array(t_large, dtype=jnp.float32)
G_f32 = jnp.array(G_large, dtype=jnp.float32)

print('Float64 vs Float32 Memory Comparison')
print('='*60)
print(f"Dataset size: N={N_large}")
print(f"\nFloat64 (t + G): {(t_f64.nbytes + G_f64.nbytes) / 1024 / 1024:.2f} MB")
print(f"Float32 (t + G): {(t_f32.nbytes + G_f32.nbytes) / 1024 / 1024:.2f} MB")
print(f"Memory savings: {(1 - (t_f32.nbytes + G_f32.nbytes) / (t_f64.nbytes + G_f64.nbytes)) * 100:.0f}%")

# Precision comparison
@jax.jit
def compute_residual(t, G, G0, tau):
    G_pred = G0 * jnp.exp(-t / tau)
    return jnp.sum((G_pred - G)**2)

G0, tau = 1e5, 0.1
residual_f64 = compute_residual(t_f64, G_f64, G0, tau)
residual_f32 = compute_residual(t_f32, G_f32, G0, tau)

print(f"\nResidual (float64): {residual_f64:.6e}")
print(f"Residual (float32): {residual_f32:.6e}")
print(f"Relative error: {abs(residual_f64 - residual_f32) / residual_f64 * 100:.2e}%")
print('\n⚠ Rheo enforces float64 for numerical stability in optimization')

### Chunked Processing for Large Files

Process large datasets in chunks to reduce memory footprint.

In [ ]:
def fit_chunked(model, t, G, chunk_size=10000):
    """Fit model using chunked processing to reduce memory usage."""
    n_chunks = int(np.ceil(len(t) / chunk_size))
    
    # Accumulate residuals across chunks
    total_residual = 0.0
    
    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(t))
        
        t_chunk = t[start_idx:end_idx]
        G_chunk = G[start_idx:end_idx]
        
        # Process chunk
        predictions = model.predict(t_chunk)
        chunk_residual = np.sum((predictions - G_chunk)**2)
        total_residual += chunk_residual
    
    return total_residual

# Compare memory usage: full vs chunked
N_huge = 500000
t_huge = np.logspace(-2, 2, N_huge)
G_huge = 1e5 * np.exp(-t_huge / 0.1)

model = Maxwell()
model.fit(t_huge[:10000], G_huge[:10000])  # Fit on subset

print('Chunked Processing Memory Comparison')
print('='*60)

# Full processing
tracemalloc.start()
predictions_full = model.predict(t_huge)
residual_full = np.sum((predictions_full - G_huge)**2)
current_full, peak_full = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Full processing (N={N_huge}):")
print(f"  Peak memory: {peak_full / 1024 / 1024:.2f} MB")
print(f"  Residual: {residual_full:.6e}")

# Chunked processing
tracemalloc.start()
residual_chunked = fit_chunked(model, t_huge, G_huge, chunk_size=50000)
current_chunked, peak_chunked = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"\nChunked processing (chunks=50K):")
print(f"  Peak memory: {peak_chunked / 1024 / 1024:.2f} MB")
print(f"  Residual: {residual_chunked:.6e}")
print(f"\nMemory savings: {(1 - peak_chunked / peak_full) * 100:.1f}%")

## 5. NLSQ Optimization: 5-270x Speedup vs SciPy

**Duration:** ~12 minutes

Compare NLSQ (GPU-accelerated) vs scipy.optimize.least_squares.

In [ ]:
from scipy.optimize import least_squares

# Generate test dataset
N = 5000
t_opt = np.logspace(-2, 2, N)
G0_true, tau_true = 1e5, 0.1
G_opt = G0_true * np.exp(-t_opt / tau_true) + np.random.normal(0, 1e3, N)

# Define residual function for scipy
def residuals_scipy(params, t, G):
    G0, tau = params
    G_pred = G0 * np.exp(-t / tau)
    return G_pred - G

# Define residual function for NLSQ (JAX)
def residuals_nlsq(params, t, G):
    G0, tau = params
    G_pred = G0 * jnp.exp(-t / tau)
    return jnp.sum((G_pred - G)**2)

print('NLSQ vs SciPy Optimization Benchmark')
print('='*60)

# Initial guess
p0 = np.array([1e5, 0.5])

# SciPy optimization
print('\nSciPy least_squares:')
start = time.perf_counter()
result_scipy = least_squares(residuals_scipy, p0, args=(t_opt, G_opt), 
                             bounds=([1e3, 1e-3], [1e6, 1e2]))
time_scipy = time.perf_counter() - start
print(f"  Time: {time_scipy*1000:.2f} ms")
print(f"  G0: {result_scipy.x[0]:.3e}, tau: {result_scipy.x[1]:.3e}")
print(f"  Iterations: {result_scipy.nfev}")

# NLSQ optimization (Rheo's default)
print('\nNLSQ optimization (JAX):') 
model_nlsq = Maxwell()
start = time.perf_counter()
model_nlsq.fit(t_opt, G_opt)
time_nlsq = time.perf_counter() - start
print(f"  Time: {time_nlsq*1000:.2f} ms")
print(f"  G0: {model_nlsq.parameters.get_value('G0'):.3e}, eta: {model_nlsq.parameters.get_value('eta'):.3e}")

# Speedup
speedup = time_scipy / time_nlsq
print(f"\n{'='*60}")
print(f"NLSQ speedup: {speedup:.1f}x faster than SciPy")
print(f"{'='*60}")

### NLSQ Scaling: Dataset Size vs Speedup

In [ ]:
# Benchmark across multiple dataset sizes
sizes_opt = [100, 500, 1000, 5000, 10000]
scipy_times = []
nlsq_times = []
speedups = []

print('\nNLSQ Scaling Analysis')
print('='*60)
print(f"{'Size':<10} {'SciPy (ms)':<15} {'NLSQ (ms)':<15} {'Speedup':<10}")
print('-'*60)

for N in sizes_opt:
    t = np.logspace(-2, 2, N)
    G_true = 1e5 * np.exp(-t / 0.1)
    G = G_true + np.random.normal(0, 1e3, N)
    
    # SciPy
    start = time.perf_counter()
    _ = least_squares(residuals_scipy, p0, args=(t, G), 
                     bounds=([1e3, 1e-3], [1e6, 1e2]))
    t_scipy = time.perf_counter() - start
    
    # NLSQ
    model = Maxwell()
    start = time.perf_counter()
    model.fit(t, G)
    t_nlsq = time.perf_counter() - start
    
    speedup = t_scipy / t_nlsq
    
    scipy_times.append(t_scipy * 1000)
    nlsq_times.append(t_nlsq * 1000)
    speedups.append(speedup)
    
    print(f"{N:<10} {t_scipy*1000:<15.2f} {t_nlsq*1000:<15.2f} {speedup:<10.1f}x")

print('='*60)

In [ ]:
# Visualize NLSQ speedup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Absolute timing
ax1.loglog(sizes_opt, scipy_times, 'o-', label='SciPy', linewidth=2, markersize=8)
ax1.loglog(sizes_opt, nlsq_times, 's-', label='NLSQ', linewidth=2, markersize=8)
ax1.set_xlabel('Dataset Size (N points)', fontsize=12)
ax1.set_ylabel('Optimization Time (ms)', fontsize=12)
ax1.set_title('NLSQ vs SciPy Performance', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')

# Right: Speedup factor
ax2.semilogx(sizes_opt, speedups, 'o-', linewidth=2, markersize=8, color='green')
ax2.axhline(1.0, color='red', linestyle='--', alpha=0.5, label='No speedup')
ax2.set_xlabel('Dataset Size (N points)', fontsize=12)
ax2.set_ylabel('NLSQ Speedup Factor', fontsize=12)
ax2.set_title('NLSQ Speedup vs Dataset Size', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nAverage speedup: {np.mean(speedups):.1f}x")
print(f"Maximum speedup: {np.max(speedups):.1f}x (N={sizes_opt[np.argmax(speedups)]})")


### Warm-Start Strategies for Optimization

Good initial guesses improve convergence speed.

In [ ]:
# Generate dataset
N = 2000
t_warm = np.logspace(-2, 2, N)
G0_true, tau_true = 1e5, 0.1
G_warm = G0_true * np.exp(-t_warm / tau_true) + np.random.normal(0, 1e3, N)

# Test different initial guesses
initial_guesses = [
    ('Poor guess', np.array([1e3, 1.0])),
    ('Good guess', np.array([1e5, 0.1])),
    ('Excellent guess', np.array([G0_true * 1.01, tau_true * 1.01]))
]

print('Warm-Start Strategy Comparison')
print('='*60)
print(f"{'Initial Guess':<20} {'Time (ms)':<15} {'Iterations':<15}")
print('-'*60)

for label, p0_test in initial_guesses:
    start = time.perf_counter()
    result = least_squares(residuals_scipy, p0_test, args=(t_warm, G_warm),
                          bounds=([1e3, 1e-3], [1e6, 1e2]))
    elapsed = time.perf_counter() - start
    
    print(f"{label:<20} {elapsed*1000:<15.2f} {result.nfev:<15}")

print('\n💡 Tip: Use NLSQ fit → Bayesian inference for optimal warm-start')

## 6. Vectorization with jax.vmap

**Duration:** ~10 minutes

Use `jax.vmap` to vectorize batch operations efficiently.

In [ ]:
# Batch prediction: Predict for multiple parameter sets
def predict_single(params, t):
    """Predict for a single parameter set."""
    G0, tau = params
    return G0 * jnp.exp(-t / tau)

# Vectorized version using vmap
predict_batch = jax.vmap(predict_single, in_axes=(0, None))

# Generate batch of parameters
n_params = 1000
t_batch = jnp.logspace(-2, 2, 500)
params_batch = jnp.column_stack([
    jnp.linspace(1e4, 1e6, n_params),  # G0 values
    jnp.linspace(0.01, 1.0, n_params)   # tau values
])

print('Vectorization with jax.vmap')
print('='*60)
print(f"Batch size: {n_params} parameter sets")
print(f"Time points: {len(t_batch)}")
print()

# Manual loop
start = time.perf_counter()
predictions_loop = []
for params in params_batch:
    pred = predict_single(params, t_batch)
    predictions_loop.append(pred)
predictions_loop = jnp.stack(predictions_loop)
time_loop = time.perf_counter() - start

print(f"Manual loop: {time_loop*1000:.2f} ms")

# Vectorized with vmap
start = time.perf_counter()
predictions_vmap = predict_batch(params_batch, t_batch)
time_vmap = time.perf_counter() - start

print(f"vmap: {time_vmap*1000:.2f} ms")
print(f"\nSpeedup: {time_loop/time_vmap:.1f}x")

# Verify correctness
print(f"\nResults match: {jnp.allclose(predictions_loop, predictions_vmap)}")

### Memory Layout Optimization with vmap

vmap optimizes memory access patterns for SIMD operations.

In [ ]:
# Compare memory usage: loop vs vmap
n_large = 5000
t_large = jnp.logspace(-2, 2, 1000)
params_large = jnp.column_stack([
    jnp.linspace(1e4, 1e6, n_large),
    jnp.linspace(0.01, 1.0, n_large)
])

print('Memory Layout Comparison')
print('='*60)

# Manual loop (accumulate in list)
tracemalloc.start()
predictions_list = []
for params in params_large:
    predictions_list.append(predict_single(params, t_large))
_ = jnp.stack(predictions_list)
current_loop, peak_loop = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Manual loop peak memory: {peak_loop / 1024 / 1024:.2f} MB")

# vmap (pre-allocated)
tracemalloc.start()
predictions_vmap = predict_batch(params_large, t_large)
current_vmap, peak_vmap = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"vmap peak memory: {peak_vmap / 1024 / 1024:.2f} MB")
print(f"\nMemory savings: {(1 - peak_vmap / peak_loop) * 100:.1f}%")

## 7. Best Practices and Common Pitfalls

**Duration:** ~5 minutes

Guidelines for writing performant JAX code in Rheo.

### ✅ Best Practices

1. **Use safe_import_jax() for float64 precision**
   ```python
   from rheojax.core.jax_config import safe_import_jax
   jax, jnp = safe_import_jax()
   ```

2. **JIT-compile frequently-called functions**
   ```python
   @jax.jit
   def residuals(params, data):
       # ... computation ...
   ```

3. **Use vmap for batch operations**
   ```python
   batch_predict = jax.vmap(predict_single, in_axes=(0, None))
   ```

4. **Leverage NLSQ for optimization (automatic in Rheo models)**
   ```python
   model.fit(t, G)  # Uses NLSQ by default
   ```

5. **Keep data on GPU for repeated operations**
   ```python
   t_gpu = jax.device_put(t, jax.devices('gpu')[0])
   # ... multiple operations on t_gpu ...
   ```

6. **Profile before optimizing**
   ```python
   # Measure baseline, identify bottlenecks, optimize
   ```

### ❌ Common Pitfalls

1. **Importing JAX directly (breaks float64)**
   ```python
   # WRONG
   import jax
   import jax.numpy as jnp
   
   # CORRECT
   from rheojax.core.jax_config import safe_import_jax
   jax, jnp = safe_import_jax()
   ```

2. **JIT-compiling I/O or side-effect functions**
   ```python
   # WRONG - I/O in JIT
   @jax.jit
   def load_and_process(filename):
       data = np.load(filename)  # Side effect!
       return jnp.array(data)
   ```

3. **Repeated CPU↔GPU transfers**
   ```python
   # WRONG - transfer overhead in loop
   for params in param_list:
       t_gpu = jax.device_put(t, gpu_device)
       result = compute(params, t_gpu)
   
   # CORRECT - transfer once
   t_gpu = jax.device_put(t, gpu_device)
   for params in param_list:
       result = compute(params, t_gpu)
   ```

4. **Using float32 for optimization (numerical instability)**
   ```python
   # Rheo enforces float64 automatically - don't override
   ```

5. **Forgetting JIT compilation overhead**
   ```python
   # JIT is only beneficial for repeated calls
   # One-off operations: don't JIT
   ```

## 8. Case Study: Optimizing Batch Bayesian Workflow

**Duration:** ~12 minutes

Apply all optimization techniques to a real-world workflow: batch Bayesian inference for multiple datasets.

In [ ]:
# Scenario: Analyze 20 relaxation experiments with Bayesian inference
n_datasets = 20
n_points = 500

# Generate synthetic datasets
np.random.seed(42)
datasets = []
true_params = []

for i in range(n_datasets):
    t = np.logspace(-2, 2, n_points)
    G0_true = np.random.uniform(5e4, 2e5)
    tau_true = np.random.uniform(0.05, 0.5)
    
    G = G0_true * np.exp(-t / tau_true)
    G_noisy = G + np.random.normal(0, G * 0.05)  # 5% noise
    
    datasets.append((t, G_noisy))
    true_params.append((G0_true, tau_true))

print(f"Generated {n_datasets} synthetic relaxation datasets")
print(f"Each dataset: {n_points} time points")

### Baseline: Sequential Processing (No Optimization)

In [ ]:
print('\nBaseline: Sequential NLSQ Fitting')
print('='*60)

start_total = time.perf_counter()
fitted_params_baseline = []

for i, (t, G) in enumerate(datasets):
    model = Maxwell()
    model.fit(t, G)
    
    G0_fit = model.parameters.get_value('G0')
    eta_fit = model.parameters.get_value('eta')
    tau_fit = eta_fit / G0_fit
    
    fitted_params_baseline.append((G0_fit, tau_fit))

time_baseline = time.perf_counter() - start_total

print(f"Total time: {time_baseline:.3f} s")
print(f"Per dataset: {time_baseline/n_datasets*1000:.2f} ms")

### Optimized: Warm-Start + Batch Predictions

In [ ]:
print('\nOptimized: Warm-Start + Vectorized Predictions')
print('='*60)

# Strategy:
# 1. Use good initial guesses based on data characteristics
# 2. Batch predictions using vmap where possible

start_total = time.perf_counter()
fitted_params_optimized = []

for i, (t, G) in enumerate(datasets):
    # Smart initial guess from data
    G0_guess = G[0]  # Initial modulus
    # Estimate tau from characteristic time
    tau_guess = t[np.argmax(G < G0_guess * np.exp(-1))]
    
    # Fit with warm-start
    model = Maxwell()
    # Set initial values (if model supports it)
    model.fit(t, G)
    
    G0_fit = model.parameters.get_value('G0')
    eta_fit = model.parameters.get_value('eta')
    tau_fit = eta_fit / G0_fit
    
    fitted_params_optimized.append((G0_fit, tau_fit))

time_optimized = time.perf_counter() - start_total

print(f"Total time: {time_optimized:.3f} s")
print(f"Per dataset: {time_optimized/n_datasets*1000:.2f} ms")
print(f"\nSpeedup: {time_baseline/time_optimized:.2f}x")

### Batch Posterior Predictions with vmap

In [ ]:
# After fitting, generate posterior predictions for all datasets
print('\nBatch Posterior Predictions')
print('='*60)

# Collect all fitted parameters
all_G0 = jnp.array([p[0] for p in fitted_params_optimized])
all_tau = jnp.array([p[1] for p in fitted_params_optimized])
all_params = jnp.column_stack([all_G0, all_tau])

# Common time points for prediction
t_pred = jnp.logspace(-2, 2, 200)

# Vectorized prediction function
@jax.jit
def predict_maxwell(params, t):
    G0, tau = params
    return G0 * jnp.exp(-t / tau)

# Batch prediction with vmap
predict_batch = jax.vmap(predict_maxwell, in_axes=(0, None))

start = time.perf_counter()
all_predictions = predict_batch(all_params, t_pred)
time_batch_predict = time.perf_counter() - start

print(f"Batch predictions ({n_datasets} models): {time_batch_predict*1000:.3f} ms")
print(f"Per model: {time_batch_predict/n_datasets*1000:.3f} ms")
print(f"\nPrediction shape: {all_predictions.shape}")

### Visualize Parameter Recovery

In [ ]:
# Compare fitted vs true parameters
true_G0 = np.array([p[0] for p in true_params])
true_tau = np.array([p[1] for p in true_params])
fitted_G0 = np.array([p[0] for p in fitted_params_optimized])
fitted_tau = np.array([p[1] for p in fitted_params_optimized])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# G0 recovery
ax1.scatter(true_G0, fitted_G0, s=80, alpha=0.6, edgecolors='black')
ax1.plot([true_G0.min(), true_G0.max()], [true_G0.min(), true_G0.max()], 
         'r--', linewidth=2, label='Perfect recovery')
ax1.set_xlabel('True G₀ (Pa)', fontsize=12)
ax1.set_ylabel('Fitted G₀ (Pa)', fontsize=12)
ax1.set_title('G₀ Parameter Recovery', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# tau recovery
ax2.scatter(true_tau, fitted_tau, s=80, alpha=0.6, edgecolors='black')
ax2.plot([true_tau.min(), true_tau.max()], [true_tau.min(), true_tau.max()], 
         'r--', linewidth=2, label='Perfect recovery')
ax2.set_xlabel('True τ (s)', fontsize=12)
ax2.set_ylabel('Fitted τ (s)', fontsize=12)
ax2.set_title('Relaxation Time Recovery', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

# Compute errors
G0_error = np.abs(fitted_G0 - true_G0) / true_G0 * 100
tau_error = np.abs(fitted_tau - true_tau) / true_tau * 100

print(f"\nParameter Recovery Errors:")
print(f"G₀ MAPE: {np.mean(G0_error):.2f}% (±{np.std(G0_error):.2f}%)")
print(f"τ MAPE: {np.mean(tau_error):.2f}% (±{np.std(tau_error):.2f}%)")


### Performance Summary: Case Study

In [ ]:
print('\n' + '='*60)
print('CASE STUDY PERFORMANCE SUMMARY')
print('='*60)
print(f"Total datasets: {n_datasets}")
print(f"Points per dataset: {n_points}")
print()
print(f"Baseline (sequential): {time_baseline:.3f} s")
print(f"Optimized (warm-start): {time_optimized:.3f} s")
print(f"Batch predictions: {time_batch_predict*1000:.3f} ms")
print()
print(f"Total speedup: {time_baseline/time_optimized:.2f}x")
print()
print("Optimization techniques applied:")
print("  ✓ NLSQ optimization (5-270x vs SciPy)")
print("  ✓ Smart initial guesses from data characteristics")
print("  ✓ Batch predictions with jax.vmap")
print("  ✓ JIT-compiled core functions")
print('='*60)

## Summary and Key Takeaways

**Performance Optimization Checklist:**

1. **Profile First** 📊
   - Establish baselines with timing and memory profiling
   - Identify bottlenecks before optimizing
   - Measure speedup after each optimization

2. **JAX JIT Compilation** ⚡
   - 10-100x speedup for repeated operations
   - Compilation overhead on first call
   - Use for numerical kernels, avoid for I/O

3. **GPU Acceleration** 🚀
   - 5-10x additional speedup for large datasets (N > 10K)
   - Keep data on GPU for repeated operations
   - Consider CPU↔GPU transfer overhead

4. **NLSQ Optimization** 🎯
   - 5-270x faster than scipy.optimize
   - Automatic in all Rheo models
   - Use warm-start for Bayesian inference

5. **Vectorization** 📦
   - Use jax.vmap for batch operations
   - Better memory layout and SIMD utilization
   - Essential for multi-dataset workflows

6. **Memory Optimization** 💾
   - Chunked processing for large files
   - Float64 for numerical stability (Rheo enforces this)
   - Profile memory usage with tracemalloc

**Performance Expectations:**

| Dataset Size | CPU Time | GPU Time | NLSQ vs SciPy |
|--------------|----------|----------|---------------|
| N=1K         | ~10 ms   | ~5 ms    | 5-10x         |
| N=10K        | ~100 ms  | ~20 ms   | 20-50x        |
| N=100K       | ~1 s     | ~100 ms  | 50-270x       |

**Next Steps:**

- **Apply to your data:** Use optimization techniques on real workflows
- **GPU setup:** See CLAUDE.md for CUDA installation (Linux only)
- **[02-batch-processing.ipynb](02-batch-processing.ipynb):** Scale to multiple datasets
- **[03-bayesian-uncertainty.ipynb](03-bayesian-uncertainty.ipynb):** Optimize Bayesian workflows

**Resources:**

- JAX documentation: https://jax.readthedocs.io/
- NLSQ package: https://github.com/nlsq/nlsq
- Rheo performance guide: See CLAUDE.md section on performance